### Import file from current directory

In [2]:
import os

In [3]:
cwd = os.getcwd()
file = 'file://{}/walmart_stock.csv'.format(cwd)

### Start a spark session

In [4]:
spark = SparkSession.builder.getOrCreate()

In [5]:
df = spark.read.csv(file, header=True, inferSchema=True)

#### What are the columns?


In [21]:
df.columns

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

#### How does the Schema look like?

In [23]:
df.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



#### Sample of the data frame

In [30]:
df.head(5)

[Row(Date=datetime.datetime(2012, 1, 3, 0, 0), Open=59.970001, High=61.060001, Low=59.869999, Close=60.330002, Volume=12668800, Adj Close=52.619234999999996),
 Row(Date=datetime.datetime(2012, 1, 4, 0, 0), Open=60.209998999999996, High=60.349998, Low=59.470001, Close=59.709998999999996, Volume=9593300, Adj Close=52.078475),
 Row(Date=datetime.datetime(2012, 1, 5, 0, 0), Open=59.349998, High=59.619999, Low=58.369999, Close=59.419998, Volume=12768200, Adj Close=51.825539),
 Row(Date=datetime.datetime(2012, 1, 6, 0, 0), Open=59.419998, High=59.450001, Low=58.869999, Close=59.0, Volume=8069400, Adj Close=51.45922),
 Row(Date=datetime.datetime(2012, 1, 9, 0, 0), Open=59.029999, High=59.549999, Low=58.919998, Close=59.18, Volume=6679300, Adj Close=51.616215000000004)]

#### Some statistical info about the Schema

In [32]:
df.describe().show()

+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|              Open|             High|              Low|            Close|           Volume|        Adj Close|
+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|              1258|             1258|             1258|             1258|             1258|             1258|
|   mean| 72.35785375357709|72.83938807631165| 71.9186009594594|72.38844998012726|8222093.481717011|67.23883848728146|
| stddev|  6.76809024470826|6.768186808159218|6.744075756255496|6.756859163732991|  4519780.8431556|6.722609449996857|
|    min|56.389998999999996|        57.060001|        56.299999|        56.419998|          2094900|        50.363689|
|    max|         90.800003|        90.970001|            89.25|        90.470001|         80898100|84.91421600000001|
+-------+------------------+-----------------+--

#### What is the ratio of the highest price vs volume traded for a day?

In [17]:
PVratioColumn = df.withColumn(colName='PriceVolumeRatio', col=(df['High'] / df['Volume']))
PriceVolumeRatio = PVratioColumn.select('PriceVolumeRatio').show()


+--------------------+
|    PriceVolumeRatio|
+--------------------+
|4.819714653321546E-6|
|6.290848613094555E-6|
|4.669412994783916E-6|
|7.367338463826307E-6|
|8.915604778943901E-6|
|8.644477436914568E-6|
|9.351828421515645E-6|
| 8.29141562102703E-6|
|7.712212102001476E-6|
|7.071764823529412E-6|
|1.015495466386981E-5|
|6.576354146362592...|
| 5.90145296180676E-6|
|8.547679455011844E-6|
|8.420709512685392E-6|
|1.041448341728929...|
|8.316075414862431E-6|
|9.721183814992126E-6|
|8.029436027707578E-6|
|6.307432259386365E-6|
+--------------------+
only showing top 20 rows



#### What day had the highest price?


In [33]:
highest = df.orderBy(df['High'].desc()).limit(1)
highest.select(highest['Date']).show()

+-------------------+
|               Date|
+-------------------+
|2015-01-13 00:00:00|
+-------------------+



#### Mean of open price

In [35]:
from pyspark.sql.functions import mean
df.select(mean('Open')).show()

+-----------------+
|        avg(Open)|
+-----------------+
|72.35785375357709|
+-----------------+



#### Max and min of volumes

In [43]:
from pyspark.sql.functions import max, min
df.select(max('Volume'), min('Volume')).show()

+-----------+-----------+
|max(Volume)|min(Volume)|
+-----------+-----------+
|   80898100|    2094900|
+-----------+-----------+



#### How many times the stocks opened above 60 dollars?

In [46]:
df.where(df['Open'] > 60).count()

1174

#### What percentage of time the stocks closed bellow 70 dollars?

In [55]:
((df.where(df['Close'] < 70).count()/df.count())*100)

31.558028616852145

#### What is the Pearson correlation between High and Volume?

In [57]:
df.corr('High', 'Volume', 'pearson')

-0.3384326061737161

#### What is the max price per year?

In [76]:
from pyspark.sql.functions import year

In [77]:
year = df.withColumn('Year', year(df['Date']))

In [83]:
year.groupBy('Year').max('High').show()

+----+---------+
|Year|max(High)|
+----+---------+
|2015|90.970001|
|2013|81.370003|
|2014|88.089996|
|2012|77.599998|
|2016|75.190002|
+----+---------+



#### What is the average close for each month?


In [84]:
from pyspark.sql.functions import month

In [85]:
monthdf = df.withColumn('Month', month('Date'))

In [94]:
monthdf.groupBy('Month').mean('Close').orderBy('Month').show()

+-----+-----------------+
|Month|       avg(Close)|
+-----+-----------------+
|    1|71.44801958415842|
|    2|  71.306804443299|
|    3|71.77794377570092|
|    4|72.97361900952382|
|    5|72.30971688679247|
|    6| 72.4953774245283|
|    7|74.43971943925233|
|    8|73.02981855454546|
|    9|72.18411785294116|
|   10|71.57854545454543|
|   11| 72.1110893069307|
|   12|72.84792478301885|
+-----+-----------------+



In [95]:
#### What is the min Peak of each day?

In [100]:
from pyspark.sql.functions import dayofmonth

In [101]:
daydf = df.withColumn('Day', dayofmonth(df['Date']))

In [102]:
daydf.groupBy('Day').min('High').orderBy('Day'.show()

+---+------------------+
|Day|         min(High)|
+---+------------------+
| 31|             61.57|
| 28|         57.720001|
| 26|         58.450001|
| 27|58.040001000000004|
| 12|             57.77|
| 22|         59.900002|
|  1|             59.27|
| 13|         57.060001|
|  6|         59.200001|
| 16|         58.029999|
|  3|         58.330002|
| 20|              59.0|
|  5|             58.98|
| 19|         59.299999|
| 15|         59.650002|
|  9|         58.779999|
| 17|60.110001000000004|
|  4|58.759997999999996|
|  8|         59.369999|
| 23|         58.900002|
+---+------------------+
only showing top 20 rows

